<a href="https://colab.research.google.com/github/ericjenn/working-groups/blob/ericjenn-srpwg-wg1/safety-related-profile/documents/profile_opset/div/div.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install onnx onnxruntime


In [46]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input1_name = "X"
input2_name = "Y"
div_output_name = "DivOutput"

#-------------------------------------------------------------------------------
# 2 rank input tensor
#-------------------------------------------------------------------------------

# Create 2-rank input tensors (two inputs for division)
input1 = onnx.helper.make_tensor_value_info(input1_name, onnx.TensorProto.FLOAT, [None, None])
input2 = onnx.helper.make_tensor_value_info(input2_name, onnx.TensorProto.FLOAT, [None, None])

# Create output tensor (final result after div operation)
div_output = onnx.helper.make_tensor_value_info(div_output_name, onnx.TensorProto.FLOAT, [None, None])

# Define division node: Z = X / Y
div_node = onnx.helper.make_node("Div", [input1_name, input2_name], [div_output_name])

# Create the ONNX graph
graph = onnx.helper.make_graph(
    nodes=[div_node],
    name="Div",
    inputs=[input1, input2],
    outputs=[div_output]
)

# Create the ONNX model
model = onnx.helper.make_model(graph, opset_imports=[onnx.helper.make_opsetid("", 13)]) # Explicitly set opset to 13
onnx.checker.check_model(model)

# Save the model
onnx.save(model, "div.onnx")

# Load and run the model using ONNX Runtime
session = ort.InferenceSession("div.onnx")

def do_div(x,y):
  # Run inference
  output = session.run(None, {input1_name: x, input2_name: y})

  x_f=(np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', ''))
  y_f=(np.array2string(y, separator=',', max_line_width=np.inf).replace('\n', ''))
  o_f=(np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', ''))

  # Display results
  print(f"X={x_f}, Y={y_f}")
  print(f"Result: X/Y={o_f}")

np.set_printoptions(precision=None, floatmode='fixed')


#--------------------------------------------------------------------
# Nominal cases
#--------------------------------------------------------------------

# Case N1: 2-rank tensor (float32)
x = np.array([[1.0, 2.0],[3.0, 4.0]], dtype=np.float32)
y = np.array([[3.0, 4.0],[5.0, 6.0]], dtype=np.float32)

# Run inference
do_div(x,y)

#--------------------------------------------------------------------
# Non nominal cases (nan and inf values)
#--------------------------------------------------------------------

v=[1, 0, float("inf"), float("nan")]

for x in v:
  for y in v:
    x_np = np.array([[x],[x]], dtype=np.float32)
    y_np = np.array([[y],[y]], dtype=np.float32)
    do_div(x_np,y_np)



X=[[1.00000000000000000000000000000000,2.00000000000000000000000000000000], [3.00000000000000000000000000000000,4.00000000000000000000000000000000]], Y=[[3.00000000000000000000000000000000,4.00000000000000000000000000000000], [5.00000000000000000000000000000000,6.00000000000000000000000000000000]]
Result: X/Y=[[0.33333334326744079589843750000000,0.50000000000000000000000000000000], [0.60000002384185791015625000000000,0.66666668653488159179687500000000]]
X=[[1.00000000000000000000000000000000], [1.00000000000000000000000000000000]], Y=[[1.00000000000000000000000000000000], [1.00000000000000000000000000000000]]
Result: X/Y=[[1.00000000000000000000000000000000], [1.00000000000000000000000000000000]]
X=[[1.00000000000000000000000000000000], [1.00000000000000000000000000000000]], Y=[[0.00000000000000000000000000000000], [0.00000000000000000000000000000000]]
Result: X/Y=[[inf], [inf]]
X=[[1.00000000000000000000000000000000], [1.00000000000000000000000000000000]], Y=[[inf], [inf]]
Result: X/Y